In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import *

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow import keras

from utils import folders_tb as f
from utils import models as mo

In [2]:
data = []
labels = []
classes = 43
cur_path = f.curr_path() + os.sep + 'data' + os.sep + "Train"
cur_path

'c:\\Users\\Karina\\Desktop\\data_science_apr_2021\\machine_learning_project\\project\\data\\Train'

In [3]:
for i in os.listdir(cur_path):
    dir = cur_path + '/' + i
    for j in os.listdir(dir):
        img_path = dir+'/'+j
        img = cv2.imread(img_path,-1)
        img = cv2.resize(img, (30,30), interpolation = cv2.INTER_NEAREST)
        data.append(img)
        labels.append(i)
        
x_train = np.array(data)
x_train = x_train / 255
y_train = np.array(labels).astype(np.float)

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

(31367, 30, 30, 3) (31367,)
(7842, 30, 30, 3) (7842,)


PROBANDO MODELO LENET5 CON EL CJTO TRAIN

In [5]:
model = mo.model_lenet_5()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 6)         456       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 6)         24        
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 16)        64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0

In [6]:
callback = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=1e-1, patience=8, verbose=1, min_lr = 2e-6)
opt = Adam(learning_rate = 1e-3)

In [7]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
history = model.fit(x_train ,y_train,  batch_size = 128, 
         epochs=2, validation_data = (x_valid, y_valid),
          callbacks=[callback, reduce_lr], shuffle = True)

Epoch 1/2
246/246 [==============================] - 11s 42ms/step - loss: 2.6244 - accuracy: 0.3408 - val_loss: 3.0212 - val_accuracy: 0.2456
Epoch 2/2
246/246 [==============================] - 10s 41ms/step - loss: 1.2742 - accuracy: 0.6521 - val_loss: 0.8680 - val_accuracy: 0.8083


In [9]:
model.save('../models/model_lenet.h5')

In [10]:
cur_path = f.curr_path() + os.sep + 'data' + os.sep + "Train"
test = pd.read_csv(f.curr_path() + os.sep + 'data' + os.sep + "Test.csv")
y_test = test["ClassId"].values
imgs = test["Path"].values
test_path = f.curr_path() + os.sep + 'data' 
data_test =[]
for img in imgs:
    image = cv2.imread(f.curr_path() + os.sep + 'data' +os.sep +img)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((30, 30))
    data_test.append(np.array( resize_image ))


In [11]:

x_test = np.array(data_test)
x_test = x_test/255
y_test = test["ClassId"].values


In [12]:
print(x_test.shape)
print(y_test.shape)

(12630, 30, 30, 3)
(12630,)


In [16]:
model_lenet5 = keras.models.load_model('../models/model_lenet.h5')

In [17]:
model_lenet5.evaluate(x_test, y_test)

395/395 [==============================] - 2s 4ms/step - loss: 1.0123 - accuracy: 0.7567


[1.0122861862182617, 0.7566904425621033]

MODELO SENCILLO DE CNN

In [20]:
x_simple_train, x_simple_valid, y_simple_train, y_simple_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=119)
print("Train set to Simple model CNN:",x_simple_train.shape, y_simple_train.shape)
print("Valid set to Simple model CNN:",x_simple_valid.shape, y_simple_valid.shape)

Train set to Simple model CNN: (21956, 30, 30, 3) (21956,)
Valid set to Simple model CNN: (9411, 30, 30, 3) (9411,)


In [21]:
x_simple_test = np.array(data_test)
x_simple_test = x_test/255
y_simple_test = test["ClassId"].values

In [23]:
print("Simple X_Test set shape:", x_simple_test.shape)
print("Simple y_Test set shape:", y_simple_test.shape)

Simple X_Test set shape: (12630, 30, 30, 3)
Simple y_Test set shape: (12630,)


In [25]:
nuevo_modelo = mo.simple_model()

AttributeError: module 'utils.models' has no attribute 'simple_model'